https://colab.research.google.com/github/Adapter-Hub/adapters/blob/main/notebooks/01_Adapter_Training.ipynb#scrollTo=hnct-N6dLvOd

In [1]:
!pip install -qq adapters datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 61.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("hrithikpiyush/acl-arc")
dataset = dataset.rename_column(original_column_name="intent", new_column_name="labels")

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1688 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/114 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/139 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/cs_roberta_base")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_train_data = train_data.map(tokenize_function, batched=True, batch_size=len(train_data))
tokenized_val_data = val_data.map(tokenize_function, batched=True, batch_size=len(val_data))
tokenized_test_data = test_data.map(tokenize_function, batched=True, batch_size=len(test_data))

tokenizer_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/1688 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [4]:
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [5]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

num_labels = len(set(train_data['labels']))

config = RobertaConfig.from_pretrained(
    "allenai/cs_roberta_base",
    num_labels=num_labels,
)
model = AutoAdapterModel.from_pretrained(
    "allenai/cs_roberta_base",
    config=config,
)

pytorch_model.bin:   0%|          | 0.00/656M [00:00<?, ?B/s]

Basically, train_adapter() method performs two functions majorly:
*   It freezes all the weights of the pre-trained model such that only the adapter weights are updated during the training.
*   It also activates the adapter and prediction head to use both in every forward pass.

In [6]:
# Add a new adapter
model.add_adapter("acl-arc", config="seq_bn")
# Alternatively, e.g.:
# model.add_adapter("rotten_tomatoes", config="lora")

# Add a matching classification head
model.add_classification_head(
    "acl-arc",
    num_labels=num_labels,
    id2label={
        0: "label1_0",
        1: "label1_1",
        2: "label1_2",
        3: "label1_3",
        4: "label1_4",
        5: "label1_5",
    }
)

# Activate the adapter
model.train_adapter("acl-arc")

In [7]:
import numpy as np
import evaluate

# https://huggingface.co/spaces/evaluate-metric/f1/blob/main/f1.py
# Example 5-A multi-label example
metric = evaluate.load("f1", "multilabel")

In [8]:
# https://medium.com/@rakeshrajpurohit/customized-evaluation-metrics-with-hugging-face-trainer-3ff00d936f99

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision_macro = precision_score(labels, preds, average='macro')
    precision_weighted = precision_score(labels, preds, average='weighted')
    recall_macro = recall_score(labels, preds, average='macro')
    recall_weighted = recall_score(labels, preds, average='weighted')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_weighted = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=5,  # <- change from 6 to 5
    per_device_train_batch_size=32,  # <- default is 8 idk what is better
    per_device_eval_batch_size=32,  # <- default is 8
    auto_find_batch_size=True,  # automates lowering the batch size to resolve out-of-memory errors
    logging_steps=100,
    output_dir="./adapter_cs_roberta_base_seq_bn_acl_arc_4",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy="epoch"
)

In [10]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
1,1.396700,1.254625,0.517544,0.086257,0.267852,0.166667,0.517544,0.113680,0.353007
2,1.211100,1.157859,0.543860,0.181524,0.381862,0.205811,0.543860,0.177726,0.426887
3,1.058800,0.991288,0.649123,0.371500,0.638453,0.311273,0.649123,0.270531,0.559878
4,1.024400,0.948345,0.666667,0.380307,0.645239,0.316922,0.666667,0.288768,0.582076
5,0.973600,0.926743,0.666667,0.310272,0.580376,0.337369,0.666667,0.306366,0.601241


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=530, training_loss=1.1218812222750683, metrics={'train_runtime': 614.7089, 'train_samples_per_second': 13.73, 'train_steps_per_second': 0.862, 'total_flos': 2259242776903680.0, 'train_loss': 1.1218812222750683, 'epoch': 5.0})

In [12]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.9267427325248718,
 'eval_accuracy': 0.6666666666666666,
 'eval_precision_macro': 0.31027238711314054,
 'eval_precision_weighted': 0.5803755607626372,
 'eval_recall_macro': 0.3373688458434221,
 'eval_recall_weighted': 0.6666666666666666,
 'eval_f1_macro': 0.30636554221459883,
 'eval_f1_weighted': 0.6012408092845035,
 'eval_runtime': 3.9681,
 'eval_samples_per_second': 28.729,
 'eval_steps_per_second': 1.008,
 'epoch': 5.0}

In [13]:
trainer.predict(tokenized_test_data)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[ 1.48975301e+00, -1.18023872e+00,  1.03687274e+00,
        -1.32107270e+00, -5.39864480e-01, -1.05343783e+00],
       [ 3.08926082e+00, -3.63164926e+00,  1.48101735e+00,
        -1.66847754e+00,  8.01398531e-02, -7.87642896e-01],
       [ 3.34444308e+00, -3.14805508e+00,  5.60727596e-01,
        -1.53936565e+00,  6.55757263e-02, -3.70668262e-01],
       [ 3.26504397e+00, -3.54371786e+00,  1.00739205e+00,
        -1.55853558e+00,  7.98011571e-02, -5.12665987e-01],
       [ 6.20926917e-01, -1.96830273e-01,  7.46925294e-01,
        -1.04133058e+00, -9.23420072e-01, -9.76481736e-01],
       [ 3.48067594e+00, -2.69063616e+00,  3.71617436e-01,
        -1.62008059e+00, -2.25617930e-01, -6.51399732e-01],
       [ 1.35271966e+00, -1.07476544e+00,  7.59939015e-01,
        -1.33992410e+00, -7.30918884e-01, -7.16265976e-01],
       [ 3.89046502e+00, -3.76680040e+00,  8.94092798e-01,
        -1.64584231e+00, -1.82432192e-03, -5.27419627e-01],
       [ 4.18246508